

5) France
===========

Sessions are present on the format: http://www.assemblee-nationale.fr/[legislature]/cri/[yearX]-[yearX+1]/[yearX]0001.asp

In [153]:
import requests
import bs4
import os

# 2001 - 2004 have some html problems
count = 0

# 11 is impossible
for i in range(12,15):
    mainPage = 'http://www.assemblee-nationale.fr'
    url = mainPage + '/' + str(i) + '/debats/index.asp'
    page = requests.get(url);
    soup = bs4.BeautifulSoup(page.text,'html.parser')

    links = []
    for element in soup.findAll('a'):
        try:
            if '/cri/' in element['href'] and 'commissions_elargies' not in element['href'] and 'congres' not in element['href']:
                links.append(mainPage + element['href'])
        except:
            pass
        
    for link in links:
        #print('looking at links from',link)
        page = requests.get(link);
        soup = bs4.BeautifulSoup(page.text,'html.parser')
        sublinks = []
        #subList = soup.find(id='listecri')
        # 2004 backwards does not work
        subList = soup.findAll(class_='seance')
        if len(subList) == 0:
            subList = soup.findAll('table',{'width':'90%'})
        for seance in subList:
            for element in seance.findAll('a'):
                try:
                    if '.asp' in element['href'] and 'index' not in element['href']:
                        if link.endswith('/'):
                            finalLink = link + element['href'].split('#')[0]
                        else:
                            finalLink = link + "/" + element['href'].split('#')[0]
                        if finalLink not in sublinks and '//' not in finalLink.split('http://')[1]:
                            sublinks.append(finalLink)
                except:
                    pass
        
        for sublink in sublinks:
            #print(sublink)
            page = requests.get(sublink);
            soup = bs4.BeautifulSoup(page.content.decode('utf-8','ignore'),'html.parser')
            
            html = soup.find(id='englobe')
            text = ''
            if html:
                text = html.text
            else:
                tables = soup.findAll('table')
                for table in tables:
                    if table['width'] == '90%':
                        text = table.text
                        break
            
            # get folder
            # get name
            temp = sublink.split("cri/")[1]
            folder = 'france/'+temp.split("/")[0]
            file = temp.split("/")[1].split(".asp")[0]
            
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            with open(folder+"/"+file+".txt","w", encoding="utf-8") as a:
                a.write(text)
                count += 1
                if count % 30 == 1:
                    print('%.2f'%(count*100/4000),"%")
                 
print("downloaded",count,"sessions!")

downloaded 4045 sessions!
